In [9]:
# №3 оценка модели. Предсказание, относится фильм к жанру Drama или нет
# только 'release_year', 'imdb_rate', 'metascore_rate', 'votes', 'duration'

import pandas as pd 
import numpy as np
import scipy.stats as sts
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import collections
np.random.seed(123)

# Отключение некоторых лишних предупреждений
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (8, 4)
        
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from nltk.probability import FreqDist

df = pd.read_csv('result_films.csv')

# Создаем список уникальных жанров
genres = set()
for genre_list in df['genre']:
    genres.update(genre_list.split(', '))


# Создаем новые столбцы для каждого жанра и заполняем их нулями
for genre in genres:
    df[genre] = 0

# Обновляем значения столбцов жанров
for index, row in df.iterrows():
    genre_list = row['genre'].split(', ')
    for genre in genre_list:
        df.at[index, genre] = 1

# Извлекаем год выпуска фильма и сохраняем его в отдельный столбец "release_year"
df['release_year'] = df['release_year'].str.extract(r'(\d{4})')

# Удаляем скобки и "(I)" из столбца "release_year"
df['release_year'] = df['release_year'].str.replace(r'\(I\)', '')

# Преобразуем столбец "release_year" в числовой формат
df['release_year'] = df['release_year'].astype(int)


# Удаление столбца "genre"
df_1 = df.drop({'name', 'global_rate','description', 'genre', 'Film-Noir', 'Family',
               'Sci-Fi', 'Adventure', 'Western', 'Crime', 'Horror', 'Comedy',
               'Romance', 'War', 'Musical', 'Mystery', 'Animation', 'Action',
               'Biography', 'History', 'Sport', 'Music', 'Fantasy', 'Thriller'}, axis=1)


# Удаление слова "min" и преобразование значения в числовой тип данных
df_1["duration"] = df_1["duration"].str.replace(" min", "")

# Замена пропущенных значений на нули
df_1["duration"].fillna(0, inplace=True)
df_1["duration"] = df_1["duration"].astype(int)

# Нормализация значений в столбцах
df_1["release_year"] = pd.to_numeric(df["release_year"], errors='coerce').astype(float)
df_1["imdb_rate"] = df_1["imdb_rate"].str.replace(',', '.').astype(float)
df_1["metascore_rate"] = pd.to_numeric(df["metascore_rate"], errors='coerce').astype(float)
df_1["votes"] = pd.to_numeric(df["votes"], errors='coerce').astype(float)

cols_to_normalize = ["release_year","imdb_rate", "metascore_rate", "votes", "duration"]
df_1[cols_to_normalize] = df_1[cols_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))


# Определение признаков и целевой переменной
features = df_1.drop(['Drama'], axis=1)  # Исключаем столбец 'Comedy'
target = df_1['Drama']


# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

numeric_data = X_train.select_dtypes([np.number])

numeric_data_mean = numeric_data.mean() 
numeric_features = numeric_data.columns

columns = X_train.columns

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)  

#нормирование данных

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])

columns = X_train.columns

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)  

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])


clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1,scoring="roc_auc" )

best_clf = clf.fit(X_train_scaled,y_train)
best_clf.best_estimator_

y_pred = best_clf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('максимизация roc-auc')
print('accuracy = ', accuracy, '; roc_auc = ',  roc_auc,'; precision = ', precision,'; recall = ', recall)

Fitting 3 folds for each of 16000 candidates, totalling 48000 fits
максимизация roc-auc
accuracy =  0.6104046242774567 ; roc_auc =  0.6070301973951538 ; precision =  0.5700280112044818 ; recall =  0.931350114416476


In [10]:
# №3 оценка модели. Предсказание, относится фильм к жанру Drama или нет
# столбцы слов из описания и тематические  (любовь, война, приключения, убийтсва),
#БЕЗ данных о рейтинге, длительности и количестве голосов

import pandas as pd 
import numpy as np
import scipy.stats as sts
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import collections
np.random.seed(123)

# Отключение некоторых лишних предупреждений
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (8, 4)
        
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from nltk.probability import FreqDist

df = pd.read_csv('result_films.csv')

# Создаем список уникальных жанров
genres = set()
for genre_list in df['genre']:
    genres.update(genre_list.split(', '))


# Считывание списка из файла в переменную
with open('unmeaning_words.txt', 'r') as file:
    unmeaning_words = file.read().splitlines()

with open('love_words.txt', 'r') as file:
    love_word_list = file.read().splitlines()

with open('death_words.txt', 'r') as file:
    death_word_list = file.read().splitlines()
    
with open('war_words.txt', 'r') as file:
    war_word_list = file.read().splitlines()
    
with open('journey_words.txt', 'r') as file:
    journey_word_list = file.read().splitlines()   

# Создаем новые столбцы для каждого жанра и заполняем их нулями
for genre in genres:
    df[genre] = 0

# Обновляем значения столбцов жанров
for index, row in df.iterrows():
    genre_list = row['genre'].split(', ')
    for genre in genre_list:
        df.at[index, genre] = 1
        
# Список с результатами
top_words_list = []

for genre in genres:
    # Фильтрация фильмов по жанру и выбор столбца 'description'
    films = df[df[genre] == 1]['description']

    # Сбор всех описаний в один текст
    descriptions = ' '.join(films)

    # Разделение текста на отдельные слова
    words = descriptions.lower().split()

    # Подсчет количества слов, за исключением слов из списка unmeaning_words
    word_counts = collections.Counter(word for word in words if word not in unmeaning_words)

    # Получение 20 самых часто встречающихся слов
    top_words = [word for word, count in word_counts.most_common(20)]

    # Сохранение результатов в список
    top_words_list.append((genre, top_words))


# Перебор элементов списка top_words_list
for genre, top_words in top_words_list:
    # Создание имени столбца
    column_name = f"word_from_{genre}"

    # Создание нового столбца с начальным значением 0
    df[column_name] = 0

    # Перебор описаний фильмов
    for index, row in df.iterrows():
        description = row['description'].lower()  # Преобразуем текст описания в нижний регистр
        
        # Проверка наличия слов из списка top_words в описании
        if any(word in description for word in top_words):
            # Установка значения 1, если хотя бы одно слово найдено
            df.loc[index, column_name] = 1

df['love_presence'] = 0
df['war_presence'] = 0
df['journey_presence'] = 0
df['death_presence'] = 0

# Проходим по каждой строке таблицы
for index, row in df.iterrows():
    description = row['description'].lower()  # Преобразуем текст описания в нижний регистр
        
    # Проверяем наличие слов из списка love_word_list
    if any(word in description for word in love_word_list):
        df.at[index, 'love_presence'] = 1
        
    # Проверяем наличие слов из списка war_word_list
    if any(word in description for word in war_word_list):
        df.at[index, 'war_presence'] = 1
        
    # Проверяем наличие слов из списка journey_word_list
    if any(word in description for word in journey_word_list):
        df.at[index, 'journey_presence'] = 1
        
    # Проверяем наличие слов из списка death_word_list
    if any(word in description for word in death_word_list):
        df.at[index, 'death_presence'] = 1
        
# Извлекаем год выпуска фильма и сохраняем его в отдельный столбец "release_year"
df['release_year'] = df['release_year'].str.extract(r'(\d{4})')

# Удаляем скобки и "(I)" из столбца "release_year"
df['release_year'] = df['release_year'].str.replace(r'\(I\)', '')

# Преобразуем столбец "release_year" в числовой формат
df['release_year'] = df['release_year'].astype(int)


# Удаление столбца "genre"
df_1 = df.drop({'name', 'global_rate', 'release_year', 'imdb_rate', 'metascore_rate',
               'description', 'votes', 'duration', 'genre', 'Film-Noir', 'Family',
               'Sci-Fi', 'Adventure', 'Western', 'Crime', 'Horror', 'Comedy',
               'Romance', 'War', 'Musical', 'Mystery', 'Animation', 'Action',
               'Biography', 'History', 'Sport', 'Music', 'Fantasy', 'Thriller'}, axis=1)

# Определение признаков и целевой переменной
features = df_1.drop(['Drama'], axis=1)  # Исключаем столбец 'Comedy'
target = df_1['Drama']


# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

numeric_data = X_train.select_dtypes([np.number])

numeric_data_mean = numeric_data.mean() 
numeric_features = numeric_data.columns

columns = X_train.columns

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)  

#нормирование данных

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])

columns = X_train.columns

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)  

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])

## подбор Solver'a
## идеи с сайта https://www.kaggle.com/code/funxexcel/p2-logistic-regression-hyperparameter-tuning

logModel = LogisticRegression()
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-5, 5, 40),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : np.logspace(0, 5, 20)
    }
]

clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1,scoring="roc_auc" )

best_clf = clf.fit(X_train_scaled,y_train)
best_clf.best_estimator_

y_pred = best_clf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('максимизация roc-auc')
print('accuracy = ', accuracy, '; roc_auc = ',  roc_auc,'; precision = ', precision,'; recall = ', recall)

Fitting 3 folds for each of 16000 candidates, totalling 48000 fits
максимизация roc-auc
accuracy =  0.6011560693641619 ; roc_auc =  0.6008121431168332 ; precision =  0.5995670995670995 ; recall =  0.6338672768878718


In [11]:
# №3 оценка модели. Предсказание, относится фильм к жанру Drama или нет
# столбцы слов из описания и тематические  (любовь, война, приключения, убийтсва),
# C данными о рейтинге, длительности и количестве голосов

import pandas as pd 
import numpy as np
import scipy.stats as sts
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import collections
np.random.seed(123)

# Отключение некоторых лишних предупреждений
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (8, 4)
        
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from nltk.probability import FreqDist

df = pd.read_csv('result_films.csv')

# Создаем список уникальных жанров
genres = set()
for genre_list in df['genre']:
    genres.update(genre_list.split(', '))


# Считывание списка из файла в переменную
with open('unmeaning_words.txt', 'r') as file:
    unmeaning_words = file.read().splitlines()

with open('love_words.txt', 'r') as file:
    love_word_list = file.read().splitlines()

with open('death_words.txt', 'r') as file:
    death_word_list = file.read().splitlines()
    
with open('war_words.txt', 'r') as file:
    war_word_list = file.read().splitlines()
    
with open('journey_words.txt', 'r') as file:
    journey_word_list = file.read().splitlines()   

# Создаем новые столбцы для каждого жанра и заполняем их нулями
for genre in genres:
    df[genre] = 0

# Обновляем значения столбцов жанров
for index, row in df.iterrows():
    genre_list = row['genre'].split(', ')
    for genre in genre_list:
        df.at[index, genre] = 1
        
# Список с результатами
top_words_list = []

for genre in genres:
    # Фильтрация фильмов по жанру и выбор столбца 'description'
    films = df[df[genre] == 1]['description']

    # Сбор всех описаний в один текст
    descriptions = ' '.join(films)

    # Разделение текста на отдельные слова
    words = descriptions.lower().split()

    # Подсчет количества слов, за исключением слов из списка unmeaning_words
    word_counts = collections.Counter(word for word in words if word not in unmeaning_words)

    # Получение 20 самых часто встречающихся слов
    top_words = [word for word, count in word_counts.most_common(20)]

    # Сохранение результатов в список
    top_words_list.append((genre, top_words))


# Перебор элементов списка top_words_list
for genre, top_words in top_words_list:
    # Создание имени столбца
    column_name = f"word_from_{genre}"

    # Создание нового столбца с начальным значением 0
    df[column_name] = 0

    # Перебор описаний фильмов
    for index, row in df.iterrows():
        description = row['description'].lower()  # Преобразуем текст описания в нижний регистр
        
        # Проверка наличия слов из списка top_words в описании
        if any(word in description for word in top_words):
            # Установка значения 1, если хотя бы одно слово найдено
            df.loc[index, column_name] = 1

df['love_presence'] = 0
df['war_presence'] = 0
df['journey_presence'] = 0
df['death_presence'] = 0

# Проходим по каждой строке таблицы
for index, row in df.iterrows():
    description = row['description'].lower()  # Преобразуем текст описания в нижний регистр
        
    # Проверяем наличие слов из списка love_word_list
    if any(word in description for word in love_word_list):
        df.at[index, 'love_presence'] = 1
        
    # Проверяем наличие слов из списка war_word_list
    if any(word in description for word in war_word_list):
        df.at[index, 'war_presence'] = 1
        
    # Проверяем наличие слов из списка journey_word_list
    if any(word in description for word in journey_word_list):
        df.at[index, 'journey_presence'] = 1
        
    # Проверяем наличие слов из списка death_word_list
    if any(word in description for word in death_word_list):
        df.at[index, 'death_presence'] = 1
            

# Извлекаем год выпуска фильма и сохраняем его в отдельный столбец "release_year"
df['release_year'] = df['release_year'].str.extract(r'(\d{4})')

# Удаляем скобки и "(I)" из столбца "release_year"
df['release_year'] = df['release_year'].str.replace(r'\(I\)', '')

# Преобразуем столбец "release_year" в числовой формат
df['release_year'] = df['release_year'].astype(int)


# Удаление столбца "genre"
df_1 = df.drop({'name', 'global_rate','description', 'genre', 'Film-Noir', 'Family',
               'Sci-Fi', 'Adventure', 'Western', 'Crime', 'Horror', 'Comedy',
               'Romance', 'War', 'Musical', 'Mystery', 'Animation', 'Action',
               'Biography', 'History', 'Sport', 'Music', 'Fantasy', 'Thriller'}, axis=1)


# Удаление слова "min" и преобразование значения в числовой тип данных
df_1["duration"] = df_1["duration"].str.replace(" min", "")

# Замена пропущенных значений на нули
df_1["duration"].fillna(0, inplace=True)
df_1["duration"] = df_1["duration"].astype(int)

# Нормализация значений в столбцах
df_1["release_year"] = pd.to_numeric(df["release_year"], errors='coerce').astype(float)
df_1["imdb_rate"] = df_1["imdb_rate"].str.replace(',', '.').astype(float)
df_1["metascore_rate"] = pd.to_numeric(df["metascore_rate"], errors='coerce').astype(float)
df_1["votes"] = pd.to_numeric(df["votes"], errors='coerce').astype(float)

cols_to_normalize = ["release_year","imdb_rate", "metascore_rate", "votes", "duration"]
df_1[cols_to_normalize] = df_1[cols_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))


# Определение признаков и целевой переменной
features = df_1.drop(['Drama'], axis=1)  # Исключаем столбец 'Comedy'
target = df_1['Drama']


# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

numeric_data = X_train.select_dtypes([np.number])

numeric_data_mean = numeric_data.mean() 
numeric_features = numeric_data.columns

columns = X_train.columns

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)  

#нормирование данных

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])

columns = X_train.columns

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)  

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])


clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1,scoring="roc_auc" )

best_clf = clf.fit(X_train_scaled,y_train)
best_clf.best_estimator_

y_pred = best_clf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('максимизация roc-auc')
print('accuracy = ', accuracy, '; roc_auc = ',  roc_auc,'; precision = ', precision,'; recall = ', recall)

Fitting 3 folds for each of 16000 candidates, totalling 48000 fits
максимизация roc-auc
accuracy =  0.7213872832369942 ; roc_auc =  0.7208157787805556 ; precision =  0.7033195020746889 ; recall =  0.7757437070938215
